In [ ]:
import drjit as dr
import mitsuba as mi
import matplotlib.pyplot as plt
import numpy as np

mi.set_variant('llvm_ad_rgb')

In [ ]:
def sph_to_dir(theta, phi):
    """Map spherical to Euclidean coordinates"""
    st, ct = dr.sincos(theta)
    sp, cp = dr.sincos(phi)
    return mi.Vector3f(cp * st, sp * st, ct)


def eval(bsdf, wo, wi):
    si = dr.zeros(mi.SurfaceInteraction3f)
    si.wi = wi
    return bsdf.eval(mi.BSDFContext(), si, wo)


def grid(bsdf, n_theta_o, n_phi_o, theta_i, phi_i, apply_cos_theta=False):
    # Create grid in spherical coordinates and map it onto the hemisphere
    theta_o, phi_o = dr.meshgrid(
        dr.linspace(mi.Float, 0, 0.5 * dr.pi, n_theta_o),
        dr.linspace(mi.Float, 0, 2 * dr.pi, n_phi_o),
    )

    # Evaluate BSDF
    value = eval(
        bsdf,
        sph_to_dir(theta_o, phi_o),
        sph_to_dir(theta_i, phi_i),
    )
    if apply_cos_theta:
        value /= dr.cos(theta_o)

    return value


def plot(
    bsdf,
    theta_i=dr.pi * 0.25,
    phi_i=0.0,
    n_theta=300,
    n_phi=1200,
    apply_cos_theta=False,
    cmap="viridis",
):
    n_theta_o = n_theta
    n_phi_o = n_phi
    values = np.array(grid(bsdf, n_theta_o, n_phi_o, theta_i, phi_i, apply_cos_theta))

    # Extract red channel of BRDF values and reshape into 2D grid
    values_plt = values[:, 0].reshape(n_phi_o, n_theta_o).T

    # Plot values for spherical coordinates
    fig, ax = plt.subplots(figsize=(16, 4))

    pixel_width = 1.0 / (n_phi + 1)
    pixel_height = 1.0 / (n_theta + 1)
    im = ax.imshow(
        values_plt,
        extent=[
            -2 * np.pi * pixel_width,
            2 * np.pi * (1 + pixel_width),
            0.5 * np.pi * (1 + pixel_height),
            -0.5 * np.pi * pixel_height,
        ],
        cmap=cmap,
    )

    ax.set_xlabel(r"$\phi_o$")
    ax.set_xticks([0, dr.pi, dr.two_pi])
    ax.set_xticklabels(["0", "$\\pi$", "$2\\pi$"])
    ax.set_ylabel(r"$\theta_o$")
    ax.set_yticks([0, 0.25 * dr.pi, 0.5 * dr.pi])
    ax.set_yticklabels(["0", "$\\pi/4$", "$\\pi / 2$"])
    plt.colorbar(im)

    return fig, ax


plot(mi.load_dict({"type": "diffuse", "reflectance": 1.0}))
plt.show()
plt.close()

plot(mi.load_dict({"type": "rpv"}), phi_i=dr.pi)
plt.show()
plt.close()


In [ ]:
def discretize(bsdf, n_theta_o, n_theta_i, n_phi_d):
    # Build list of incident and outgoing directions
    cos_theta_os = dr.linspace(mi.Float, 0.0, 1.0, n_theta_o)
    theta_os = dr.acos(cos_theta_os)
    cos_theta_is = dr.linspace(mi.Float, 0.0, 1.0, n_theta_i)
    theta_is = dr.acos(cos_theta_is)
    phi_os = dr.linspace(mi.Float, 0.0, dr.two_pi, n_phi_d)
    phi_i = mi.Float(0.0)

    # Generate vector coordinate lists
    theta_ov, phi_ov, theta_iv, phi_iv =  dr.meshgrid(
        theta_os,
        phi_os, 
        theta_is,
        phi_i,
        indexing="ij"
    )
    wos = sph_to_dir(theta_ov, phi_ov)
    wis = sph_to_dir(theta_iv, phi_iv)

    # Evaluate BSDF
    si = dr.zeros(mi.SurfaceInteraction3f)
    si.wi = wis
    return bsdf.eval(mi.BSDFContext(), si, wos) / dr.cos(theta_ov)

In [ ]:
n_theta_o, n_theta_i, n_phi_d = 9, 3, 36

diffuse = mi.load_dict({"type": "diffuse"})
plot(diffuse, n_theta=n_theta_o, n_phi=n_phi_d)
plt.show()
plt.close()

rpv = mi.load_dict({"type": "rpv"})
plot(rpv, n_theta=n_theta_o, n_phi=n_phi_d)
plt.show()
plt.close()

In [ ]:
n_theta_o, n_theta_i, n_phi_d = 9, 3, 36

data = np.array(
    discretize(diffuse, n_theta_o, n_theta_i, n_phi_d)
)[:, 0]
print(data.shape)
data = np.reshape(data, (n_phi_d, n_theta_i, n_theta_o), order="C")
data = np.reshape(data, (n_theta_o, n_theta_i, n_phi_d), order="F")
data.shape


In [ ]:
mqd = mi.load_dict(
    {
        "type": "measuredquasidiffuse",
        "grid": mi.VolumeGrid(data),
    }
)
mqd


In [ ]:
plot(mqd, n_theta=n_theta_o, n_phi=n_phi_d);

In [ ]:
def check_grid(n_cos_theta_o, n_phi_d, n_cos_theta_i):
    return np.reshape(
        [
            (k * n_phi_d + j) * n_cos_theta_o + i
            for k in range(n_cos_theta_i)
            for j in range(n_phi_d)
            for i in range(n_cos_theta_o)
        ],
        (n_cos_theta_i, n_phi_d, n_cos_theta_o),
        order="C",
    ).astype(np.float32)

size = 2, 3, 2
data = check_grid(*size)
print(data.shape)
print(data)
print(data.ravel())


plot(
    mi.load_dict(
        {
            "type": "measuredquasidiffuse",
            "grid": mi.VolumeGrid(data),
        }
    ),
    n_theta=size[0],
    n_phi=size[1],
    # theta_i=0.0,
    # phi_i=0.0,
)

In [ ]:
# Eval test
import numpy as np
import drjit as dr
import mitsuba as mi

mi.set_variant("scalar_rgb")


def sph_to_dir(theta, phi):
    """Map spherical to Euclidean coordinates"""
    st, ct = dr.sincos(theta)
    sp, cp = dr.sincos(phi)
    return mi.Vector3f(cp * st, sp * st, ct)


data = np.array(
    [
        # cos_theta_i = 0
        [
            # phi_d = 0
            np.linspace(0, 1, 5),
            # phi_d = π
            -np.linspace(0, 1, 5),
            # phi_d = 2π
            np.linspace(0, 1, 5),
        ],
        # cos_theta_i = 1
        [
            # phi_d = 0
            np.linspace(1, 2, 5),
            # phi_d = π
            -np.linspace(1, 2, 5),
            # phi_d = 2π
            np.linspace(1, 2, 5),
        ],
    ],
)
display(data)
display(data.ravel())
print(f"{data.shape = }")

bsdf = mi.load_dict(
    {
        "type": "measuredquasidiffuse",
        "grid": mi.VolumeGrid(data),
    }
)
print(bsdf)
params = mi.traverse(bsdf)
print(params["data"])
print(np.ravel(params["data"]))

print()

for (theta_o, phi_o), (theta_i, phi_i), expected in [
    [(np.pi / 3, 0.0), (0.0, 0.0), 1.5],
    [(np.pi * 0.4195693767448338, 0.0), (0.0, 0.0), 1.25],  # Such that cos θ = 0.25
    [(np.pi / 3, np.pi), (0.0, 0.0), -1.5],
    [(np.pi / 3, 0.5 * np.pi), (0.0, 0.0), 0.0],
    [(np.pi / 3, 1.5 * np.pi), (0.0, 0.0), 0.0],
    [(np.pi / 2, 0.0), (np.pi / 2, 0.0), 0.0],
    [(np.pi / 3, 0.0), (np.pi / 2, 0.0), 0.5],
    [(np.pi / 3, np.pi), (np.pi / 2, 0.0), -0.5],
    [(np.pi / 2, np.pi), (np.pi / 2, 0.0), 0.0],
    [(np.pi / 2, np.pi), (0.0, 0.0), -1.0],
]:
    wo = sph_to_dir(theta_o, phi_o)
    wi = sph_to_dir(theta_i, phi_i)
    print(f"{wo = }")
    print(f"{wi = }")

    si = dr.zeros(mi.SurfaceInteraction3f)
    si.wi = wi
    value = bsdf.eval(mi.BSDFContext(), si, wo)[0]

    if not dr.allclose(value, expected * dr.cos(theta_o), atol=1e-6):
        print("** Failed **")
        print(f"{(theta_o, phi_o), (theta_i, phi_i) = }")
        print(f"{expected = }")
        print(f"{value = }")

    print()


In [ ]:
mi.set_variant("llvm_ad_rgb")

bsdf = mi.load_dict(
    {
        "type": "measuredquasidiffuse",
        "grid": mi.VolumeGrid(data),
    }
)

plot(bsdf, cmap="coolwarm");

In [ ]:
np.arccos(0.25) / np.pi

In [ ]:
0.1875 * 4 * 4 * 3
3 / 16
0.1875 / 0.25
0.25 * 3 / 4